In [1]:
#Saturated Oxygen Data Correction
from oxyfloat import OxyFloat, RequiredVariableNotPresent, OpenDAPServerError
of = OxyFloat()

In [2]:
oga_floats = of.get_oxy_floats()

In [3]:
# Testing with a float that has data
oga_floats = ['1900650']
test_file = 'D1900650_137.nc'

In [4]:
for dac_url in of.get_dac_urls(oga_floats):
    for profile_url in of.get_profile_opendap_urls(dac_url):
        try:
            if not profile_url.endswith(test_file):
                continue
        except NameError:
            pass
        print 'Reading data from ...' + profile_url[40:]
        try:
            d = of.get_profile_data(profile_url)
        except RequiredVariableNotPresent as e:
            print e
        except OpenDAPServerError as e:
            print e

Reading data from ...IOLIS-ARGO-GDAC-OBS/coriolis/1900650/profiles/D1900650_137.nc


In [5]:
# Demonstrate reading the data
print d.keys()
print d['p'][0]['long_name'] + ' (' + d['p'][0]['units'] + ')'
print d['p'][1][:10]
print d['o'][0]['long_name'] + ' (' + d['o'][0]['units'] + ')'
print d['o'][1][:10]

['lon', 'o', 'p', 's', 't', 'lat', 'dt']
SEA PRESSURE (decibar)
[  4.30000019   5.9000001    9.10000038  13.90000057  19.5         24.39999962
  29.5         34.5         39.59999847  44.59999847]
DISSOLVED OXYGEN (micromole/kg)
[ 206.49000549  206.38000488  206.30000305  206.8500061   206.86000061
  211.50999451  218.30999756  232.38000488  216.80000305  190.08000183]


In [8]:
print d['o'][0].keys()

['comment', '_FillValue', 'C_format', 'resolution', 'FORTRAN_format', 'long_name', 'valid_min', 'units', 'valid_max']


The code below is form the old notebook. Some of it may be folded into oxyfloat functions.

In [6]:
#The p here is for depth...maybe
#That's why it said the unit was km
def dens_formula(p , t , s):
    C = (999.83 + (5.053*p) - (.048*(p**2)))
    B =  .808 - (.0085*p)
    a = .0708*((1 + (.351*p)) + .068*(1 - (.0683*p))*t)
    y = .003*(1 - (.059*p) - .012*(1 - .064*p)*t)
    dens = C + (B*s) - (a*t) - ((y*(35 - s))*t)
    return dens

In [7]:
dens_formula(p , t , s)

NameError: name 'p' is not defined

In [ ]:
def dens_formula(p , t , s):
    p = p / 10.0
    pw = 999.842594 + ((6.793952 * (10**-abs(2)))*t) - ((9.095290 * (10**-abs(3)))*(t**2)) + ((1.001685 * (10**-abs(4)))*(t**3)) - ((1.120083 * (10**-abs(6)))*(t**4)) + ((6.536332 * (10**-abs(9))*(t**5))        
    